In [1]:
import numpy as np
import matplotlib as mpl
%matplotlib inline

import pandas as pd

import json
import glob
import os

import seaborn

In [2]:
data = {}

for path in glob.glob("../../data/results/all_real/*.json") + \
    glob.glob("../../data/results/all_real_seq/*.json") + \
    glob.glob("../../data/results/all_real_seq2/*.json") + \
    glob.glob("../../data/results/gossip_map/*.json"):
  for typename, items in json.load(open(path)).items():
      if typename in data:
        for key, object_data in items.items():
          if key in data[typename]:
            data[typename][key].update(object_data)
          else:
            data[typename][key] = object_data
      else:
        data[typename] = items

frames = { typename: pd.DataFrame.from_dict(items, orient='index') for typename, items in data.items() }

In [3]:
dlslm_label = 'DSLM-Mod'
dlslm_me_label = 'DSLM-Map'
seq_postfix = ' w. Seq.'
no_contraction_postfix = ' w/o Contraction'
dlslm_ws_label = dlslm_label + seq_postfix
dlslm_nc_label = dlslm_label + no_contraction_postfix
seq_louvain_label = 'Seq. Louvain'
seq_infomap_label = 'Seq. InfoMap'
plm_label = 'PLM'
relax_map_label = 'RelaxMap'
gossip_map_label = 'GossipMap'

algo_name_mapping = {
    'synchronous local moving with map equation': dlslm_me_label,
    'synchronous local moving with modularity': dlslm_label,
    'sequential louvain': seq_louvain_label,
    'sequential infomap': seq_infomap_label,
    'relax map': relax_map_label,
    'gossip map': gossip_map_label
}

frames['algorithm_run'].replace({ 'algorithm': algo_name_mapping }, inplace=True)

frames['algorithm_run']['algorithm'] += frames['algorithm_run'].merge(frames['program_run'], left_on='program_run_id', right_index=True, how='left')['switch_to_seq'].map({ False: '', True: seq_postfix, np.NaN: '' })
frames['algorithm_run']['algorithm'] += frames['algorithm_run'].merge(frames['program_run'], left_on='program_run_id', right_index=True, how='left')['contraction'].map({ False: no_contraction_postfix, True: '', np.NaN: '' })

In [4]:
frames['algorithm_run']['runtime'].fillna((frames['algorithm_run']['done_ts'] - frames['algorithm_run']['start_ts']) / 1000000.0, inplace=True)

In [5]:
frames['program_run']['graph_path'] = frames['program_run']['graph']

graph_names = { 
    'data/graphs/uk-2002.metis-preprocessed-*.bin': 'uk-2002', 
    'data/graphs/uk-2007-05.metis-preprocessed-*.bin': 'uk-2007-05', 
    'data/graphs/in-2004.metis-preprocessed-*.bin': 'in-2004', 
    'data/graphs/com-friendster-preprocessed-*.bin': 'friendster', 
    'data/graphs/com-lj.ungraph-preprocessed-*.bin': 'lj', 
    'data/graphs/com-orkut.ungraph-preprocessed-*.bin': 'orkut', 
    'data/graphs/com-youtube.ungraph-preprocessed-*.bin': 'youtube', 
    'data/graphs/com-amazon.ungraph-preprocessed-*.bin': 'amazon',
    'data/graphs/europe.osm-preprocessed-*.bin': 'osm-europe',
}

frames['program_run'].replace({ 'graph': graph_names }, inplace=True)

In [14]:
all_data = frames['clustering'] \
    .merge(frames['algorithm_run'], left_on='algorithm_run_id', right_index=True) \
    .merge(frames['program_run'], left_on='program_run_id', right_index=True) \
    .groupby(['graph', 'algorithm'])['runtime'].mean().round(0).to_frame() \
    .unstack()["runtime"][[seq_louvain_label, plm_label, dlslm_label, dlslm_nc_label, seq_infomap_label, relax_map_label, gossip_map_label, dlslm_me_label]]

all_data = all_data.loc[frames['program_run'].sort_values('edge_count')['graph'].dropna().unique()]

graph_data = frames['program_run'].dropna(subset=['hosts', 'edge_count']).groupby('graph').agg({ 'node_count': 'first', 'edge_count': 'first', 'hosts': 'max' })
graph_data['hosts'] = graph_data['hosts'].astype(int)
graph_data['edge_count'] = graph_data['edge_count'].astype(int)
graph_data.columns = ['n', 'm', 'hosts']
res = graph_data.loc[['uk-2002', 'uk-2007-05', 'friendster',  'lj', 'orkut']].sort_values('m').merge(all_data, left_index=True, right_index=True)

# with open("../../dist-thrill-cluster/plots/real_world_runtimes.tex", "w") as file:
print(res.to_latex().replace('.0', '').replace('  NaN', 'oom'))
res

\begin{tabular}{lrrrrrrrrrrr}
\toprule
{} &          n &           m &  hosts &  Seq. Louvain &     PLM &  DSLM-Mod &  DSLM-Mod w/o Contraction &  Seq. InfoMap &  RelaxMap &  GossipMap &  DSLM-Map \\
graph      &            &             &        &               &         &           &                           &               &           &            &           \\
\midrule
lj         &    3997962 &    34681189 &      8 &          99 &    25 &      31 &                      14 &        1329 &     163 &      372 &      49 \\
orkut      &    3072441 &   117185083 &      8 &         170 &    53 &      47 &                      34 &        2405 &     415 &      700 &      84 \\
uk-2002    &   18483186 &   261787258 &      8 &         572 &   142 &      46 &                      22 &        6656 &     240 &      682 &      52 \\
friendster &   65608366 &  1806067135 &     16 &        6002 &  1755 &    1047 &                     742 &         oom &     oom &    13743 &    1161 \\
uk-2007-05

,n,m,hosts,Seq. Louvain,PLM,DSLM-Mod,DSLM-Mod w/o Contraction,Seq. InfoMap,RelaxMap,GossipMap,DSLM-Map
graph,,,,,,,,,,,
lj,3997962,34681189,8,99.0,25.0,31.0,14.0,1329.0,163.0,372.0,49.0
orkut,3072441,117185083,8,170.0,53.0,47.0,34.0,2405.0,415.0,700.0,84.0
uk-2002,18483186,261787258,8,572.0,142.0,46.0,22.0,6656.0,240.0,682.0,52.0
friendster,65608366,1806067135,16,6002.0,1755.0,1047.0,742.0,NaN,NaN,13743.0,1161.0
uk-2007-05,105153952,3301876564,16,7993.0,2520.0,151.0,106.0,NaN,NaN,4211.0,214.0
